In [1]:
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq
model = ChatGroq(model="llama-3.1-8b-instant")

In [2]:
def multiply(a:int,b:int)->int:
    """Muultiply a and b

    Args:
        a (int): First int
        b (int): Second int

    
    """
    return a*b

def add(a:int,b:int)->int:
    """Addition of a and b

    Args:
        a (int): First int
        b (int): Second int

   
    """
    return a+b

def divide(a:int,b:int)->int:
    """Divide a and b

    Args:
        a (int): First int
        b (int): Second int
   
    """
    return a/b

tools = [add,divide,multiply]  
tools  

[<function __main__.add(a: int, b: int) -> int>,
 <function __main__.divide(a: int, b: int) -> int>,
 <function __main__.multiply(a: int, b: int) -> int>]

In [3]:
##First Way to pass as a List
llm_with_tools = model.bind_tools([add,divide,multiply])
llm_with_tools

llm_with_tools.invoke("What is ai")


##Another way
#llm_with_tools = model.bind_tools(tools)
#tools



AIMessage(content='I\'m unable to provide a definition for "ai" as it can refer to various things such as Artificial Intelligence, American Idol, etc. If you could provide more context, I can try to provide a more accurate answer.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 361, 'total_tokens': 407, 'completion_time': 0.080091106, 'completion_tokens_details': None, 'prompt_time': 0.019953873, 'prompt_tokens_details': None, 'queue_time': 0.050188496, 'total_time': 0.100044979}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b2bbc-e592-7b02-8c32-f25aa935468f-0', usage_metadata={'input_tokens': 361, 'output_tokens': 46, 'total_tokens': 407})

In [4]:
from langchain_core.messages import HumanMessage,AIMessage
from langgraph.graph import MessagesState


st_msg = AIMessage(content="you are helpful to provide the assistance of arthmetic operations")

def assistance(state:MessagesState):
    return {"messages":[llm_with_tools.invoke([st_msg] + state["messages"])]}

def human_feedback(state:MessagesState):
    pass

In [ ]:
from langgraph.graph import START,END,StateGraph
from IPython.display import Image,display,SVG
from langgraph.prebuilt import ToolNode,tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables.graph import MermaidDrawMethod

memory = MemorySaver()

###Define the Graph
graph = StateGraph(MessagesState)

## Define the Nodes
graph.add_node("human_feedback",human_feedback)
graph.add_node("tools",ToolNode(tools))
graph.add_node("assistance",assistance)

## Define the Edges
graph.add_edge(START,"human_feedback")
graph.add_edge("human_feedback","assistance")
graph.add_conditional_edges(
    "assistance",
    tools_condition,
    
)
graph.add_edge("tools","human_feedback")

graph_builder = graph.compile(interrupt_before=['human_feedback'],checkpointer=memory)

display(Image(graph_builder.get_graph().draw_mermaid_png()))

ValueError: Failed to reach https://mermaid.ink API while trying to render your graph. Status code: 400.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [10]:
print(graph_builder.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	human_feedback(human_feedback<hr/><small><em>__interrupt = before</em></small>)
	tools(tools)
	assistance(assistance)
	__end__([<p>__end__</p>]):::last
	__start__ --> human_feedback;
	assistance -.-> __end__;
	assistance -.-> tools;
	human_feedback --> assistance;
	tools --> human_feedback;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [ ]:
# ===============================
# 1. Load environment variables
# ===============================
from dotenv import load_dotenv
load_dotenv()

# ===============================
# 2. Fix Jupyter asyncio issue
# ===============================
import nest_asyncio
nest_asyncio.apply()

# ===============================
# 3. Create LLM
# ===============================
from langchain_groq import ChatGroq

model = ChatGroq(
    model="llama-3.1-8b-instant"
)

# ===============================
# 4. Define tools
# ===============================
def add(a: int, b: int) -> int:
    """Addition"""
    return a + b

def multiply(a: int, b: int) -> int:
    """Multiplication"""
    return a * b

def divide(a: int, b: int) -> float:
    """Division"""
    return a / b

tools = [add, multiply, divide]

llm_with_tools = model.bind_tools(tools)

# ===============================
# 5. LangGraph imports
# ===============================
from langchain_core.messages import AIMessage
from langgraph.graph import StateGraph, START
from langgraph.graph import MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# ===============================
# 6. Define nodes
# ===============================
system_msg = AIMessage(
    content="You are a helpful assistant for arithmetic operations."
)

def assistant(state: MessagesState):
    response = llm_with_tools.invoke(
        [system_msg] + state["messages"]
    )
    return {"messages": [response]}

def human_feedback(state: MessagesState):
    # Human-in-the-loop interrupt
    pass

# ===============================
# 7. Build graph
# ===============================
memory = MemorySaver()
graph = StateGraph(MessagesState)

graph.add_node("human_feedback", human_feedback)
graph.add_node("assistant", assistant)
graph.add_node("tools", ToolNode(tools))

graph.add_edge(START, "human_feedback")
graph.add_edge("human_feedback", "assistant")
graph.add_conditional_edges("assistant", tools_condition)
graph.add_edge("tools", "human_feedback")

graph_builder = graph.compile(
    interrupt_before=["human_feedback"],
    checkpointer=memory
)

# ===============================
# 8. Draw Mermaid graph (Jupyter-safe)
# ===============================
from IPython.display import Image, display
from langchain_core.runnables.graph import MermaidDrawMethod

display(
    Image(
        graph_builder.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.PYPPETEER
        )
    )
)
